In [2]:
import sys
sys.path.append('../_amigocloud')

from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

## REPORTE
from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

#from docx2pdf import convert
import os

In [3]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [4]:
ruta

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [5]:
def generar_reporte(i):
    id_reg = i
    
    # SELECCIONAR REGISTRO
    query = {'query': 'select * from dataset_318090 where id = {id_ca}'.format(id_ca=id_reg)}
    project = 'https://app.amigocloud.com/api/v1/projects/33032/sql'
    select = amigocloud.get(project, query)
    data = select['data'][0]
    data['fecha_de_registro'] = datetime.strptime(data['fecha_de_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
    if data['observaciones'] == None:
        data['observaciones']='Sin observaciones'
    insp = collections.namedtuple("insp", data.keys())(*data.values())
    print('CAÑERO: ' + insp.canhero)
    #FOTOS
    query_fotos = {'query': 'select source_amigo_id, s3_filename from gallery_49781 where source_amigo_id=\'{source_amigo_id}\''.format(source_amigo_id=insp.amigo_id)}
    project = 'https://app.amigocloud.com/api/v1/projects/33032/sql'
    fotos = amigocloud.get(project, query_fotos)['data']
    #fotos = [{'source_amigo_id': 'f8d0380f6a4346479dcc15ceeaaae53a', 's3_filename': 'IMG-20231101-WA0049.jpg'}]
    #GENERAR DOCX
    doc = DocxTemplate(ruta + "/templates/tpl_rpt_operador-cosechadora.docx")
    #descargar fotos y generar lista InlineImage
    lista_fotos_inline = []
    for foto in fotos:
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
        file = open(ruta + '/fotos/' + foto['s3_filename'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor=ruta + '/fotos/' + foto['s3_filename'], width=Mm(60))})

    context = {'insp':insp, 'fotos':lista_fotos_inline}

    doc.render(context)

    # formato de nombre de archivo: "123_EVC_01-01-2022_NOMBRE"
    cod_nom = insp.canhero.split(' / ')
    file_name = ruta + '/informes/_' + str(cod_nom[0]) + '_ECO_' + insp.fecha_de_registro.replace('/','-') + '_' + cod_nom[1] + '_' + str(insp.id)

    #doc.save(r'C:\Users\UTEA\OneDrive - Ingenio Azucarero Guabira S.A (2)\INFORMES\IMFORMES PARA REVISAR\\' + file_name + '.docx')
    doc.save(file_name + '.docx')

In [7]:
lista_id = list(range(491,493))
lista_id

[491, 492]

In [12]:
for i in lista_id:
    print('GENERANDO id = ' + str(i))
    generar_reporte(i)
print('END=====================================')

GENERANDO id = 491
CAÑERO: 633 / AGROP. VICENTE ROCA GIL SRL
GENERANDO id = 492
CAÑERO: 8014 / NAZER PARADA ROBERTO DOMINGO
END=====================================


## CONVERTIR A PDF

In [8]:
path = r'' + ruta + '/informes'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

SE ENCONTRARON 4 INFORMES
4 INFORMES


['D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/75_ECO_19-06-2023_AGUILERA TARADELLES MARIANO_193.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/8014_ECO_20-06-2023_NAZER PARADA ROBERTO DOMINGO_194.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/805_ECO_19-06-2023_AGROP. TRES HNOS DEL ORIENTE_192.docx',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/86_ECO_19-06-2023_AGUILERA TARADELLES JOSE LUIS_191.docx']

In [9]:
i = 1
for filename in rutas:
    print('CONVIRTIENDO... INFORME ' + str(i) + ' ' + filename)
    # se realiza la convrsion pasando parametro de salida y entrada, 
    # para la salida se reemplaza la extencion y la ruta
    print(filename)
    print(filename.replace('.docx','.pdf').replace('informes','pdfs'))
    pdf = filename.replace('.docx','.pdf').replace('informes','pdfs').split('/')[-1]
    convert(filename, r'C:\Users\UTEA\Downloads\\' + pdf)
    i+=1
print('END')

CONVIRTIENDO... INFORME 1 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/75_ECO_19-06-2023_AGUILERA TARADELLES MARIANO_193.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/75_ECO_19-06-2023_AGUILERA TARADELLES MARIANO_193.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/75_ECO_19-06-2023_AGUILERA TARADELLES MARIANO_193.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 2 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/8014_ECO_20-06-2023_NAZER PARADA ROBERTO DOMINGO_194.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/8014_ECO_20-06-2023_NAZER PARADA ROBERTO DOMINGO_194.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/8014_ECO_20-06-2023_NAZER PARADA ROBERTO DOMINGO_194.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 3 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/805_ECO_19-06-2023_AGROP. TRES HNOS DEL ORIENTE_192.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/805_ECO_19-06-2023_AGROP. TRES HNOS DEL ORIENTE_192.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/805_ECO_19-06-2023_AGROP. TRES HNOS DEL ORIENTE_192.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

CONVIRTIENDO... INFORME 4 D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/86_ECO_19-06-2023_AGUILERA TARADELLES JOSE LUIS_191.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/86_ECO_19-06-2023_AGUILERA TARADELLES JOSE LUIS_191.docx
D:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/86_ECO_19-06-2023_AGUILERA TARADELLES JOSE LUIS_191.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

END


## CARGAR REPORTES A DRIVE

In [12]:
from docx2pdf import convert
import os

In [13]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]
ruta

'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [14]:
# colocar aqui la ruta de los archivos
path = ruta + r'/informes_eval_cosechadoras_operativo'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

SE ENCONTRARON 46 INFORMES
46 INFORMES


['D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes_eval_cosechadoras_operativo/10697_ECO_06-07-2023_PAZ REA JAIME EDUARDO_214.pdf',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes_eval_cosechadoras_operativo/11171_ECO_09-06-2023_QUIROZ SARAVIA TEODOCIA_188.pdf',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes_eval_cosechadoras_operativo/12075_ECO_05-06-2023_ROJAS GONZALES TOMAS_182.pdf',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes_eval_cosechadoras_operativo/12075_ECO_06-07-2023_ROJAS GONZALES TOMAS_215.pdf',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes_eval_cosechadoras_operativo/12075_ECO_22-05-2023_ROJAS GONZALES TOMAS_168.pdf',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes_eval_cosechadoras_operativo/12075_ECO_27-05-2023_ROJAS GONZALES TOMAS_176.pdf',
 'D:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes_eval_cosechadoras_o

In [1]:
# CONEXION A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1096764895697-57btfmtn7rdejufct5qq4j9mi61j0unl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [3]:
import os

In [4]:
contenido = os.listdir(ruta + r'/informes_eval_cosechadoras_operativo')
dic_files = {}
for file in contenido:
    split = file.split('_')
    dic_files[int(split[0])] = file
print(len(dic_files), 'informes')
dic_files

NameError: name 'ruta' is not defined